<h2><center>Iris dataset</center></h2>
<div><center>Rozpoznávání druhu květiny Iris podle rozměrů okvětních lístků.</center></div>
<hr>

In [1]:
// Instalace Nuget packagů
#r "nuget: Microsoft.Data.Analysis"
    
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: MathNet.Numerics"
    
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.AutoML"
#r "nuget: Microsoft.ML.DataView"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages MathNet.Numerics, 5.0.0 Microsoft.Data.Analysis, 0.19.1 Microsoft.ML, 1.7.1 Microsoft.ML.AutoML, 0.19.1 Microsoft.ML.DataView, 1.7.1 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [2]:
// Načtení namespaců
using System;
using System.Linq;
using System.IO;
using System.Globalization;

using XPlot.Plotly;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

using MathNet.Numerics.Statistics;

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;
using Microsoft.ML.Trainers;

In [3]:
// Formátovač tabulek - použít jen v případě, že se data nezobrazují v tabulce
/*
Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 10;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");
*/

In [4]:
// Třída pro načtení dat knihovnou Microsoft.ML
class ModelInput
{
    [LoadColumn(1)]
    public float SepalLengthCm { get; set; }

    [LoadColumn(2)]
    public float SepalWidthCm { get; set; }

    [LoadColumn(3)]
    public float PetalLengthCm { get; set; }

    [LoadColumn(4)]
    public float PetalWidthCm { get; set; }

    [LoadColumn(5)]
    public string Species { get; set; }
}

In [5]:
class ModelOutput
{
    public uint Label { get; set; }

    public float[] Features { get; set; }

    public uint PredictedLabel { get; set; }

    public float[] Score { get; set; }
}

In [6]:
// Názvy sloupců
var columnNames = new List<string>() { "SepalLengthCm", "SepalWidthCm", "PetalLengthCm",
                                      "PetalWidthCm", "Species" };
// Vytváření modelu
var context = new MLContext();

// Načtení dat knihovnou Microsoft.ML
var data = context.Data.LoadFromTextFile<ModelInput>("./Dataset/Iris.csv", hasHeader: true, separatorChar: ',');
var df = data.ToDataFrame(maxRows: -1);

In [7]:
// Zobrazení dat
df

index,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [8]:
// Typ dat každého sloupce a počet položek ve sloupci
df.Info()

index,Info,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,DataType,System.Single,System.Single,System.Single,System.Single,System.String
1,Length (excluding null values),150,150,150,150,150


In [9]:
// Základní popis dat
df.Description()

index,Description,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,Length (excluding null values),150,150,150,150
1,Max,7.9,4.4,6.9,2.5
2,Min,4.3,2,1,0.1
3,Mean,5.8433347,3.054,3.7586665,1.1986669


In [10]:
// Zobrazí všechny sloupce a jejich popis
df.Columns

index,type,value
0,Microsoft.Data.Analysis.SingleDataFrameColumn,"[ 5.1, 4.9, 4.7, 4.6, 5, 5.4, 4.6, 5, 4.4, 4.9, 5.4, 4.8, 4.8, 4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1 ... (more) ]"
1,Microsoft.Data.Analysis.SingleDataFrameColumn,"[ 3.5, 3, 3.2, 3.1, 3.6, 3.9, 3.4, 3.4, 2.9, 3.1, 3.7, 3.4, 3, 3, 4, 4.4, 3.9, 3.5, 3.8, 3.8 ... (more) ]"
2,Microsoft.Data.Analysis.SingleDataFrameColumn,"[ 1.4, 1.4, 1.3, 1.5, 1.4, 1.7, 1.4, 1.5, 1.4, 1.5, 1.5, 1.6, 1.4, 1.1, 1.2, 1.5, 1.3, 1.4, 1.7, 1.5 ... (more) ]"
3,Microsoft.Data.Analysis.SingleDataFrameColumn,"[ 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.2, 0.2, 0.1, 0.2, 0.2, 0.1, 0.1, 0.2, 0.4, 0.4, 0.3, 0.3, 0.3 ... (more) ]"
4,Microsoft.Data.Analysis.StringDataFrameColumn,"[ Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa, Iris-setosa ... (more) ]"


In [11]:
// Spočte položky v zadaném sloupci
df.Columns["Species"].ValueCounts()

index,Values,Counts
0,Iris-setosa,50
1,Iris-versicolor,50
2,Iris-virginica,50


In [12]:
// Kontrola nullových hodnot v každém sloupci
List<long> nullCount = new List<long>();

for (int i = 0; i < columnNames.Count; i++)
{
   nullCount.Add(df.Columns[columnNames[i]].NullCount);
}

PrimitiveDataFrameColumn<long> nullCountCol = new PrimitiveDataFrameColumn<long>("Null Count", nullCount);
StringDataFrameColumn colNames = new StringDataFrameColumn("Column name", columnNames);

DataFrame nullCountDf = new DataFrame(colNames, nullCountCol);
nullCountDf

index,Column name,Null Count
0,SepalLengthCm,0
1,SepalWidthCm,0
2,PetalLengthCm,0
3,PetalWidthCm,0
4,Species,0


<h4><center>Vizualizace dat</center></h4>
<hr>

In [13]:
// Funkce na tvoření histogramů
List<PlotlyChart> histogramCreator(DataFrame df, List<string> columnName)
{
    List<PlotlyChart> histograms = new List<PlotlyChart>();

    for (int i = 0; i < columnName.Count; i++)
    {
        var hist = Chart.Plot(
            new XPlot.Plotly.Histogram
            {
                x = df.Columns[columnName[i]]
            }
        );

        var hist_layout = new Layout.Layout
        {
            title = columnName[i],
            xaxis = new Xaxis
            {
                title = columnName[i]
            },
            yaxis = new Yaxis
            {
                title = "Quantity"
            }
        };

        hist.WithLayout(hist_layout);
        histograms.Add(hist);
    }

    return histograms;
}

In [14]:
// Funkce na tvoření scatter plotů
PlotlyChart scatterCreator(DataFrame df, string columnNameX, string columnNameY)
{
    var scatter = Chart.Plot(
        new Scatter
        {
            x = df.Columns[columnNameX],
            y = df.Columns[columnNameY],
            mode = "markers",
            marker = new Marker
            {
                color = "rgb(164, 194, 244)", // Barva markeru
                size = 12, // Velikost markeru
                line = new Line // Border markeru
                {
                    color = "blue",
                    width = 0.5
                }
            }
        }
    );

    var scatter_layout = new Layout.Layout
    {
        title = columnNameX + " and " + columnNameY,
        xaxis = new Xaxis
        {
            title = columnNameX
        },
        yaxis = new Yaxis
        {
            title = columnNameY
        }
    };

    scatter.WithLayout(scatter_layout);

    return scatter;
}

In [15]:
// Funkce na tvoření box plotů
PlotlyChart BoxPlotCreator(DataFrameColumn y, string columnName)
{
    var boxPlot = Chart.Plot(
        new Box
        {
            y = y,
            marker = new Marker()
            {
                color = "#3D9970"
            }
        }
    );

    var boxPlot_layout = new Layout.Layout
    {
        title = columnName,
        xaxis = new Xaxis
        {
            title = columnName
        }
    };

    boxPlot.WithLayout(boxPlot_layout);
    return boxPlot;
}

In [16]:
// Zobrazí všechny histogramy
var histograms = histogramCreator(df, columnNames);
Chart.ShowAll(histograms)

In [17]:
// Zobrazí všechny scatter ploty
var scatters = new List<PlotlyChart>();

for (int i = 0; i < columnNames.Count; i++)
{
    for (int j = 0; j < columnNames.Count; j++)
    {
        if(columnNames[i] != columnNames[j])
        {
            scatters.Add(scatterCreator(df, columnNames[i], columnNames[j]));
        }
    }
}
Chart.ShowAll(scatters)

In [18]:
// Vizualizace pomocí box plotů
var boxPlotList = new List<PlotlyChart>();
for(int i = 0; i < columnNames.Count; i++)
{
    boxPlotList.Add(BoxPlotCreator(df.Columns[columnNames[i]], columnNames[i]));
}
Chart.ShowAll(boxPlotList);

In [19]:
// Scatter plot
// Vezme všechny řádky, které obsahují iris-setosu
// A z těchto řádků vezme první sloupec
var setosaRows = df.OrderBy("Species").Rows.Where(row => row.Last().ToString() == "Iris-setosa");
var setosaSepalLength = setosaRows.Select(x => x[0]).ToList();
var setosaSepalWidth = setosaRows.Select(x => x[1]).ToList();

var versicolorRows = df.OrderBy("Species").Rows.Where(row => row.Last().ToString() == "Iris-versicolor");
var versicolorSepalLength = versicolorRows.Select(x => x[0]).ToList();
var versicolorSepalWidth = versicolorRows.Select(x => x[1]).ToList();

var virginicaRows = df.OrderBy("Species").Rows.Where(row => row.Last().ToString() == "Iris-virginica");
var virginicaSepalLength = virginicaRows.Select(x => x[0]).ToList();
var virginicaSepalWidth = virginicaRows.Select(x => x[1]).ToList();

var scatterPlots = new List<Scatter>{
    new Scatter
    {
        x = setosaSepalLength,
        y = setosaSepalWidth,        
        mode = "markers",
        name = "Iris-setosa",
        marker = new Marker
        {
            color = "rgb(164, 194, 244)", // Barva markeru
            size = 12, // Velikost markeru
            line = new Line // Border markeru
            {
                color = "blue",
                width = 0.5
            }
        }
    },
    new Scatter
    {
        x = versicolorSepalLength,
        y = versicolorSepalWidth,        
        mode = "markers",
        name = "Iris-versicolor",
        marker = new Marker
        {
            color = "red", 
            size = 12, 
            line = new Line 
            {
                color = "orange",
                width = 0.5
            }
        }
    },
    new Scatter
    {
        x = virginicaSepalLength,
        y = virginicaSepalWidth,        
        mode = "markers",
        name = "Iris-virginica",
        marker = new Marker
        {
            color = "yellow",
            size = 12, 
            line = new Line 
            {
                color = "orange",
                width = 0.5
            }
        }
    }
};

var scatterPlots_layout = new Layout.Layout{
    title = "Sepal Length and Sepal Width in Cm",
    xaxis = new Xaxis{
        title = "Length"
    },
    yaxis =new Yaxis{
        title = "Width"
    }
};

// Je nutné na list zavolat funkci plot a výsledek uložit do proměnné
var scatterPlot = Chart.Plot(scatterPlots);
scatterPlot.WithLayout(scatterPlots_layout);
scatterPlot

<!DOCTYPE html>

In [20]:
// Scatter plot
// Vezme všechny řádky, které obsahují iris-setosu
// A z těchto řádků vezme první sloupec
var setosaRows = df.OrderBy("Species").Rows.Where(row => row.Last().ToString() == "Iris-setosa");
var setosaPetalLength = setosaRows.Select(x => x[2]).ToList();
var setosaPetalWidth = setosaRows.Select(x => x[3]).ToList();

var versicolorRows = df.OrderBy("Species").Rows.Where(row => row.Last().ToString() == "Iris-versicolor");
var versicolorPetalLength = versicolorRows.Select(x => x[2]).ToList();
var versicolorPetalWidth = versicolorRows.Select(x => x[3]).ToList();

var virginicaRows = df.OrderBy("Species").Rows.Where(row => row.Last().ToString() == "Iris-virginica");
var virginicaPetalLength = virginicaRows.Select(x => x[2]).ToList();
var virginicaPetalWidth = virginicaRows.Select(x => x[3]).ToList();

var scatterPlots = new List<Scatter>{
    new Scatter
    {
        x = setosaPetalLength,
        y = setosaPetalWidth,        
        mode = "markers",
        name = "Iris-setosa",
        marker = new Marker
        {
            color = "rgb(164, 194, 244)", // Barva markeru
            size = 12, // Velikost markeru
            line = new Line // Border markeru
            {
                color = "blue",
                width = 0.5
            }
        }
    },
    new Scatter
    {
        x = versicolorPetalLength,
        y = versicolorPetalWidth,        
        mode = "markers",
        name = "Iris-versicolor",
        marker = new Marker
        {
            color = "red", 
            size = 12, 
            line = new Line 
            {
                color = "orange",
                width = 0.5
            }
        }
    },
    new Scatter
    {
        x = virginicaPetalLength,
        y = virginicaPetalWidth,        
        mode = "markers",
        name = "Iris-virginica",
        marker = new Marker
        {
            color = "yellow",
            size = 12, 
            line = new Line 
            {
                color = "orange",
                width = 0.5
            }
        }
    }
};

var scatterPlots_layout = new Layout.Layout{
    title = "Petal Length and Petal Width in Cm",
    xaxis = new Xaxis{
        title = "Length"
    },
    yaxis =new Yaxis{
        title = "Width"
    }
};

// Je nutné na list zavolat funkci plot a výsledek uložit do proměnné
var scatterPlot = Chart.Plot(scatterPlots);
scatterPlot.WithLayout(scatterPlots_layout);
scatterPlot

<!DOCTYPE html>

In [21]:
// Změní sloupec stringů na sloupec floatů
List<Single> StringConverter(List<string> stringList)
{
    var strings = new List<string>();
    var floatList = new List<Single>();

    foreach(string s in stringList)
    {
        if (!strings.Contains(s))
        {
            strings.Add(s);
            floatList.Add((float)strings.IndexOf(s));
        }
        else
        {
            floatList.Add((float)strings.IndexOf(s));
        }
    }
    
    return floatList;
}

In [22]:
// Převede specifický sloupec, který byl typu string na typ Single
var corrDf = df.Clone();
var stringList = df.Columns["Species"].Cast<string>().ToList();
var floatList = StringConverter(stringList);
// Odstranění požadovaného sloupce se string hodnotami a připojení nového sloupce již se změněnými hodnotami 
corrDf.Columns.Remove("Species");
PrimitiveDataFrameColumn<Single> newColumn = new PrimitiveDataFrameColumn<Single>("Species", floatList);
corrDf.Columns.Add(newColumn);
corrDf

index,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5,3.6,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


In [23]:
// Převedení sloupců z dataframu na pole
var matrixColumns = new double[columnNames.Count][];
for(int i = 0; i < columnNames.Count; i++)
{
     matrixColumns[i] = corrDf.Columns[columnNames[i]].Cast<Single>().ToList().ConvertAll(x => (double)x).ToArray();
}

In [24]:
// Vypočítání korelační matice
var corrMatrix = Correlation.PearsonMatrix(matrixColumns);

// Převedení matice na data frame kvůli vizualizaci tabulky
var doubleDataFrameColumn = new List<DoubleDataFrameColumn>();
for(int i = 0; i < columnNames.Count; i++)
{
    doubleDataFrameColumn.Add(new DoubleDataFrameColumn(columnNames[i], corrMatrix.Column(i)));
}

DataFrame corrMatrixDf = new DataFrame(doubleDataFrameColumn);
corrMatrixDf

index,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,-0.10936928622571292,0.8717541656669878,0.8179536373532039,0.7825612451594246
1,-0.10936928622571292,1,-0.42051611476913897,-0.35654410088264593,-0.4194462149834242
2,0.8717541656669878,-0.42051611476913897,1,0.9627570943307198,0.9490425460506304
3,0.8179536373532039,-0.35654410088264593,0.9627570943307198,1,0.9564638214620798
4,0.7825612451594246,-0.4194462149834242,0.9490425460506304,0.9564638214620798,1


In [25]:
// Vizualizace korelační matice
Chart.Plot(
    new Heatmap
    {
        x = columnNames,
        y = columnNames,
        z = corrMatrix.ToColumnArrays()
    }
)

<!DOCTYPE html>

<h4><center>Trénování modelu</center></h4>
<hr>

In [26]:
// Rozdělení datasetu na trénovací a testovací množiny 70:30
DataOperationsCatalog.TrainTestData split = context.Data.TrainTestSplit(data, testFraction: 0.3);

string[] featureColumnNames = data.Schema.Select(column => column.Name)
                          .Where(columnName => columnName != "Species").ToArray();

// Tvorba pipeliny 
IEstimator<ITransformer> pipeline = context.Transforms.Concatenate("Features", featureColumnNames)
    .Append(context.Transforms.Conversion.MapValueToKey("Label", "Species")) // Změní string hodnoty na int
    .Append(context.MulticlassClassification.Trainers.LbfgsMaximumEntropy());

ITransformer model = pipeline.Fit(split.TrainSet);

var predictions = model.Transform(split.TestSet);

// Převedení IDataView na List
var listOfPredictions = context.Data.CreateEnumerable<ModelOutput>(predictions, reuseRowObject: false).ToList();

// Zobrazení náhodných 5 predikcí
Random rand = new Random();
foreach (var p in listOfPredictions.OrderBy(x => rand.Next()).Take(5))
{
    Console.WriteLine($"Label: {p.Label}, Prediction: {p.PredictedLabel}\n");
}

var metrics = context.MulticlassClassification.Evaluate(predictions);

Console.WriteLine($"Micro Accuracy: {metrics.MicroAccuracy}");
Console.WriteLine($"Macro Accuracy: {metrics.MacroAccuracy}");
Console.WriteLine($"Log Loss: {metrics.LogLoss}");

Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());

Label: 1, Prediction: 1

Label: 2, Prediction: 2

Label: 3, Prediction: 3

Label: 1, Prediction: 1

Label: 2, Prediction: 2

Micro Accuracy: 1
Macro Accuracy: 1
Log Loss: 0.16332898995944975

Confusion table
                   ||========================
PREDICTED          ||     0 |     1 |     2 | Recall
TRUTH              ||========================
0.     Iris-setosa ||    14 |     0 |     0 | 1.0000
1. Iris-versicolor ||     0 |    12 |     0 | 1.0000
2.  Iris-virginica ||     0 |     0 |    14 | 1.0000
                   ||========================
Precision          ||1.0000 |1.0000 |1.0000 |




warning CS1701: Předpokládá se, že odkaz na sestavení Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60, který používá Microsoft.DotNet.Interactive.Formatting, odpovídá identitě Microsoft.AspNetCore.Html.Abstractions, Version=6.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60 pro Microsoft.AspNetCore.Html.Abstractions. Možná budete muset zadat zásady pro běh.



<h4><center>Validace modelu náhodných vzorkem</center></h4>
<hr>

In [27]:
// Nový testovací vzorek 
var irisTestData = new ModelInput()
{
    SepalLengthCm = 5.0f,
    SepalWidthCm = 3.4f,
    PetalLengthCm = 1.7f,
    PetalWidthCm = 0.2f
};

var predictionEngine = context.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);

var testDataPrediction = predictionEngine.Predict(irisTestData);

StringDataFrameColumn labelCol = new StringDataFrameColumn("Label", new string[] { "Iris-setosa",
                                                "Iris-versicolor", "Iris-virginica"});
StringDataFrameColumn predCol = new StringDataFrameColumn("Prediction", new string[] { $"{testDataPrediction.Score[0]*100:F2}%",
                                                 $"{testDataPrediction.Score[1]*100:F2}%",  $"{testDataPrediction.Score[2]*100:F2}%" });

DataFrame predictionDf = new DataFrame(labelCol, predCol);
predictionDf

index,Label,Prediction
0,Iris-setosa,92.58%
1,Iris-versicolor,7.42%
2,Iris-virginica,0.00%


<h4><center>Auto ML </center></h4>
<div><center>Automaticky natrénovaný model s nejlepší přesností</center></div>
<hr>

In [28]:
// AutoML
var experiment = context.Auto().CreateMulticlassClassificationExperiment(maxExperimentTimeInSeconds: 10);

var result = experiment.Execute(data, labelColumnName: "Species");

Console.WriteLine(result.BestRun.TrainerName);

Console.WriteLine(result.BestRun.ValidationMetrics.MacroAccuracy);

SdcaMaximumEntropyMulti
0.9772727272727273


<h4><center>Ukládání a načítaní modelů</center></h4>
<hr>

In [30]:
// Uložení modelu do uvedené složky
string modelName = "IrisModel";
context.Model.Save(model, data.Schema, $"./Models/{modelName}.zip");

In [31]:
// Definice DataViewSchema pro přípravu pipeliny a trénování dat
DataViewSchema modelSchema;

// Načtení modelu z uvedené složky
ITransformer trainedModel = context.Model.Load($"./Models/{modelName}.zip", out modelSchema);

PredictionEngine<ModelInput, ModelOutput> predictionEngine = context.Model.CreatePredictionEngine<ModelInput, ModelOutput>(trainedModel);

// Vzorek dat
ModelInput inputData = new ModelInput
{
    SepalLengthCm = 5.0f,
    SepalWidthCm = 3.2f,
    PetalLengthCm = 1.3f,
    PetalWidthCm = 0.2f
};

// Predikce načteného modelu
ModelOutput prediction = predictionEngine.Predict(inputData);

// Vizualizace predikce
StringDataFrameColumn labelCol = new StringDataFrameColumn("Label", new string[] { "Iris-setosa",
                                                "Iris-versicolor", "Iris-virginica"});
StringDataFrameColumn predCol = new StringDataFrameColumn("Prediction", new string[] { $"{testDataPrediction.Score[0]*100:F2}%",
                                                 $"{testDataPrediction.Score[1]*100:F2}%",  $"{testDataPrediction.Score[2]*100:F2}%" });

DataFrame predictionDf = new DataFrame(labelCol, predCol);
predictionDf

index,Label,Prediction
0,Iris-setosa,92.58%
1,Iris-versicolor,7.42%
2,Iris-virginica,0.00%
